# Simulação da Distribuição Binomial

Este notebook apresenta simulações práticas e interativas da **Distribuição Binomial**, permitindo que você experimente e visualize os conceitos em ação.

## 📚 Revisão Teórica

A **distribuição binomial** modela situações onde:
- Realizamos **n** experimentos independentes
- Cada experimento tem apenas **2 resultados possíveis** (sucesso ou fracasso)
- A **probabilidade de sucesso** é constante (**p**) em todos os experimentos
- Queremos calcular a probabilidade de obter exatamente **k** sucessos

### Fórmula:
$$P(X = k) = \binom{n}{k} \cdot p^k \cdot (1-p)^{n-k}$$

### Propriedades:
- **Média**: $E[X] = n \cdot p$
- **Variância**: $Var(X) = n \cdot p \cdot (1-p)$
- **Desvio Padrão**: $\sigma = \sqrt{n \cdot p \cdot (1-p)}$

## 🛠️ Importando as Bibliotecas

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import binom
import pandas as pd

# Configuração dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Bibliotecas importadas com sucesso!")

## 🎯 Exemplo 1: Simulando Lançamentos de Moeda

Vamos começar com um exemplo clássico: **lançar uma moeda 10 vezes** e contar quantas vezes sai "cara".

- **n = 10** (número de lançamentos)
- **p = 0.5** (probabilidade de "cara")
- **k** pode variar de 0 a 10 (número de "caras")

In [ ]:
# Parâmetros do exemplo
n = 10  # número de lançamentos
p = 0.5  # probabilidade de cara

# Valores possíveis de k (sucessos)
k_valores = np.arange(0, n + 1)

# Calculando as probabilidades para cada valor de k
probabilidades = binom.pmf(k_valores, n, p)

# Propriedades teóricas
media_teorica = n * p
variancia_teorica = n * p * (1 - p)
desvio_teorico = np.sqrt(variancia_teorica)

print(f"📊 Propriedades Teóricas:")
print(f"   Média: {media_teorica:.2f} caras")
print(f"   Variância: {variancia_teorica:.2f}")
print(f"   Desvio Padrão: {desvio_teorico:.2f}")
print()
print(f"📈 Probabilidades:")
for i, prob in enumerate(probabilidades):
    print(f"   P(X = {i:2d}) = {prob:.4f} ({prob*100:5.2f}%)")

In [ ]:
# Visualização da distribuição
plt.figure(figsize=(12, 6))

# Gráfico de barras
plt.subplot(1, 2, 1)
plt.bar(k_valores, probabilidades, alpha=0.8, color='skyblue', edgecolor='navy')
plt.axvline(media_teorica, color='red', linestyle='--', linewidth=2, label=f'Média = {media_teorica:.1f}')
plt.xlabel('Número de Caras (k)')
plt.ylabel('Probabilidade P(X = k)')
plt.title('Distribuição Binomial (n=10, p=0.5)\nLançamento de Moedas')
plt.legend()
plt.grid(True, alpha=0.3)

# Gráfico de probabilidades acumuladas
plt.subplot(1, 2, 2)
prob_acumuladas = binom.cdf(k_valores, n, p)
plt.plot(k_valores, prob_acumuladas, 'o-', color='orange', linewidth=2, markersize=6)
plt.xlabel('Número de Caras (k)')
plt.ylabel('Probabilidade Acumulada P(X ≤ k)')
plt.title('Função de Distribuição Acumulada')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🧪 Exemplo 2: Simulação com Dados Reais

Agora vamos **simular** o experimento 1000 vezes e comparar com a teoria:

In [ ]:
# Simulação: realizar o experimento 1000 vezes
num_simulacoes = 1000
np.random.seed(42)  # Para reprodutibilidade

# Gerando dados simulados
resultados_simulados = binom.rvs(n, p, size=num_simulacoes)

# Estatísticas da simulação
media_simulada = np.mean(resultados_simulados)
desvio_simulado = np.std(resultados_simulados)
variancia_simulada = np.var(resultados_simulados)

print(f"🔬 Resultados da Simulação ({num_simulacoes} experimentos):")
print(f"   Média Simulada: {media_simulada:.2f} caras")
print(f"   Desvio Simulado: {desvio_simulado:.2f}")
print(f"   Variância Simulada: {variancia_simulada:.2f}")
print()
print(f"📊 Comparação Teoria vs Simulação:")
print(f"   Média - Teoria: {media_teorica:.2f} | Simulação: {media_simulada:.2f} | Diferença: {abs(media_teorica - media_simulada):.2f}")
print(f"   Desvio - Teoria: {desvio_teorico:.2f} | Simulação: {desvio_simulado:.2f} | Diferença: {abs(desvio_teorico - desvio_simulado):.2f}")

In [ ]:
# Comparação visual: Teoria vs Simulação
plt.figure(figsize=(14, 6))

# Histograma dos resultados simulados vs teoria
plt.subplot(1, 2, 1)
frequencias_sim, _, _ = plt.hist(resultados_simulados, bins=np.arange(-0.5, n+1.5, 1), 
                                 alpha=0.7, color='lightgreen', density=True, 
                                 label='Simulação', edgecolor='black')
plt.bar(k_valores, probabilidades, alpha=0.6, color='red', 
        label='Teoria', width=0.8)
plt.xlabel('Número de Caras (k)')
plt.ylabel('Probabilidade')
plt.title('Comparação: Teoria vs Simulação')
plt.legend()
plt.grid(True, alpha=0.3)

# Convergência das médias
plt.subplot(1, 2, 2)
medias_acumuladas = np.cumsum(resultados_simulados) / np.arange(1, num_simulacoes + 1)
plt.plot(range(1, num_simulacoes + 1), medias_acumuladas, color='blue', alpha=0.7)
plt.axhline(media_teorica, color='red', linestyle='--', linewidth=2, label=f'Média Teórica = {media_teorica}')
plt.xlabel('Número de Simulações')
plt.ylabel('Média Acumulada')
plt.title('Convergência para a Média Teórica\n(Lei dos Grandes Números)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎲 Exemplo 3: Efeito dos Parâmetros n e p

Vamos explorar como diferentes valores de **n** (número de tentativas) e **p** (probabilidade de sucesso) afetam a forma da distribuição:

In [ ]:
# Configurações para comparação
configs = [
    {'n': 10, 'p': 0.1, 'label': 'n=10, p=0.1', 'color': 'blue'},
    {'n': 10, 'p': 0.5, 'label': 'n=10, p=0.5', 'color': 'green'},
    {'n': 10, 'p': 0.8, 'label': 'n=10, p=0.8', 'color': 'red'},
    {'n': 30, 'p': 0.2, 'label': 'n=30, p=0.2', 'color': 'orange'},
]

plt.figure(figsize=(15, 10))

for i, config in enumerate(configs):
    n_conf = config['n']
    p_conf = config['p']
    
    # Valores de k possíveis
    k_vals = np.arange(0, n_conf + 1)
    
    # Probabilidades
    probs = binom.pmf(k_vals, n_conf, p_conf)
    
    # Propriedades
    media = n_conf * p_conf
    desvio = np.sqrt(n_conf * p_conf * (1 - p_conf))
    
    # Subplot para cada configuração
    plt.subplot(2, 2, i + 1)
    plt.bar(k_vals, probs, alpha=0.7, color=config['color'], edgecolor='black')
    plt.axvline(media, color='red', linestyle='--', linewidth=2, 
                label=f'μ={media:.1f}, σ={desvio:.1f}')
    plt.xlabel('Número de Sucessos (k)')
    plt.ylabel('Probabilidade')
    plt.title(f'Distribuição Binomial\n{config["label"]}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Limitar eixo x para melhor visualização
    if n_conf > 20:
        plt.xlim(max(0, media - 3*desvio), min(n_conf, media + 3*desvio))

plt.tight_layout()
plt.show()

## 🏭 Exemplo 4: Aplicação Prática - Controle de Qualidade

**Cenário**: Uma fábrica produz componentes eletrônicos. Sabemos que 5% dos componentes são defeituosos. Em um lote de 20 componentes, qual a probabilidade de encontrar exatamente 2 defeituosos?

In [ ]:
# Parâmetros do problema
n_componentes = 20
p_defeituoso = 0.05
k_defeituosos = 2

# Cálculo da probabilidade específica
prob_exatos_2 = binom.pmf(k_defeituosos, n_componentes, p_defeituoso)

# Probabilidades de interesse para controle de qualidade
prob_0_defeituosos = binom.pmf(0, n_componentes, p_defeituoso)
prob_1_defeituoso = binom.pmf(1, n_componentes, p_defeituoso)
prob_ate_1_defeituoso = binom.cdf(1, n_componentes, p_defeituoso)
prob_2_ou_mais = 1 - prob_ate_1_defeituoso

print("🏭 ANÁLISE DE CONTROLE DE QUALIDADE")
print("=" * 40)
print(f"Lote: {n_componentes} componentes")
print(f"Taxa de defeito: {p_defeituoso*100}%")
print()
print("📊 PROBABILIDADES:")
print(f"   P(exatamente {k_defeituosos} defeituosos) = {prob_exatos_2:.4f} ({prob_exatos_2*100:.2f}%)")
print(f"   P(nenhum defeituoso) = {prob_0_defeituosos:.4f} ({prob_0_defeituosos*100:.2f}%)")
print(f"   P(1 defeituoso) = {prob_1_defeituoso:.4f} ({prob_1_defeituoso*100:.2f}%)")
print(f"   P(até 1 defeituoso) = {prob_ate_1_defeituoso:.4f} ({prob_ate_1_defeituoso*100:.2f}%)")
print(f"   P(2 ou mais defeituosos) = {prob_2_ou_mais:.4f} ({prob_2_ou_mais*100:.2f}%)")
print()

# Média e desvio esperados
media_defeituosos = n_componentes * p_defeituoso
desvio_defeituosos = np.sqrt(n_componentes * p_defeituoso * (1 - p_defeituoso))

print("📈 ESTATÍSTICAS ESPERADAS:")
print(f"   Número médio de defeituosos: {media_defeituosos:.2f}")
print(f"   Desvio padrão: {desvio_defeituosos:.2f}")

In [ ]:
# Visualização da distribuição para o problema de controle de qualidade
k_vals_cq = np.arange(0, 8)  # Focar nos valores mais prováveis
probs_cq = binom.pmf(k_vals_cq, n_componentes, p_defeituoso)

plt.figure(figsize=(12, 6))

# Gráfico principal
plt.subplot(1, 2, 1)
cores = ['lightgreen' if k <= 1 else 'orange' if k == 2 else 'red' for k in k_vals_cq]
bars = plt.bar(k_vals_cq, probs_cq, alpha=0.8, color=cores, edgecolor='black')

# Destacar a barra de k=2
bars[2].set_color('gold')
bars[2].set_edgecolor('red')
bars[2].set_linewidth(3)

plt.axvline(media_defeituosos, color='red', linestyle='--', linewidth=2, 
            label=f'Média = {media_defeituosos:.1f}')
plt.xlabel('Número de Componentes Defeituosos')
plt.ylabel('Probabilidade')
plt.title('Controle de Qualidade - Distribuição Binomial\n(n=20, p=0.05)')
plt.legend()
plt.grid(True, alpha=0.3)

# Adicionar anotações
plt.annotate(f'P(X=2) = {prob_exatos_2:.3f}', 
            xy=(2, prob_exatos_2), xytext=(4, prob_exatos_2 + 0.05),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=12, fontweight='bold')

# Simulação para verificação
plt.subplot(1, 2, 2)
simulacao_cq = binom.rvs(n_componentes, p_defeituoso, size=5000)
plt.hist(simulacao_cq, bins=np.arange(-0.5, 8.5, 1), alpha=0.7, 
         density=True, color='lightblue', edgecolor='black', label='Simulação')
plt.bar(k_vals_cq, probs_cq, alpha=0.6, color='red', width=0.6, label='Teoria')
plt.xlabel('Número de Componentes Defeituosos')
plt.ylabel('Probabilidade')
plt.title('Verificação por Simulação\n(5000 lotes)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análise da simulação
freq_2_defeituosos = np.sum(simulacao_cq == 2) / len(simulacao_cq)
print(f"\n🔬 VERIFICAÇÃO POR SIMULAÇÃO:")
print(f"   Frequência de exatamente 2 defeituosos: {freq_2_defeituosos:.4f} ({freq_2_defeituosos*100:.2f}%)")
print(f"   Diferença da teoria: {abs(freq_2_defeituosos - prob_exatos_2):.4f}")

## 🎯 Exercício Interativo: Teste seus Conhecimentos

Modifique os parâmetros abaixo e observe como a distribuição muda:

In [ ]:
# MODIFIQUE ESTES PARÂMETROS E EXECUTE A CÉLULA
n_exercicio = 15     # Número de tentativas (0 < n ≤ 50)
p_exercicio = 0.3    # Probabilidade de sucesso (0 < p < 1)
k_interesse = 5      # Número de sucessos de interesse (0 ≤ k ≤ n)

# Validação dos parâmetros
if not (0 < n_exercicio <= 50):
    print("⚠️ ERRO: n deve estar entre 1 e 50")
elif not (0 < p_exercicio < 1):
    print("⚠️ ERRO: p deve estar entre 0 e 1")
elif not (0 <= k_interesse <= n_exercicio):
    print(f"⚠️ ERRO: k deve estar entre 0 e {n_exercicio}")
else:
    # Cálculos
    k_vals_ex = np.arange(0, n_exercicio + 1)
    probs_ex = binom.pmf(k_vals_ex, n_exercicio, p_exercicio)
    
    prob_k_interesse = binom.pmf(k_interesse, n_exercicio, p_exercicio)
    prob_ate_k = binom.cdf(k_interesse, n_exercicio, p_exercicio)
    
    media_ex = n_exercicio * p_exercicio
    desvio_ex = np.sqrt(n_exercicio * p_exercicio * (1 - p_exercicio))
    
    # Resultados
    print("🎯 RESULTADOS DO SEU EXERCÍCIO:")
    print("=" * 35)
    print(f"Parâmetros: n={n_exercicio}, p={p_exercicio}")
    print(f"Média esperada: {media_ex:.2f}")
    print(f"Desvio padrão: {desvio_ex:.2f}")
    print()
    print(f"P(X = {k_interesse}) = {prob_k_interesse:.4f} ({prob_k_interesse*100:.2f}%)")
    print(f"P(X ≤ {k_interesse}) = {prob_ate_k:.4f} ({prob_ate_k*100:.2f}%)")
    print(f"P(X > {k_interesse}) = {1-prob_ate_k:.4f} ({(1-prob_ate_k)*100:.2f}%)")
    
    # Gráfico
    plt.figure(figsize=(10, 6))
    cores_ex = ['gold' if k == k_interesse else 'lightblue' for k in k_vals_ex]
    plt.bar(k_vals_ex, probs_ex, alpha=0.8, color=cores_ex, edgecolor='navy')
    plt.axvline(media_ex, color='red', linestyle='--', linewidth=2, 
                label=f'Média = {media_ex:.1f}')
    
    # Destacar k de interesse
    plt.annotate(f'P(X={k_interesse})\n= {prob_k_interesse:.3f}', 
                xy=(k_interesse, prob_k_interesse), 
                xytext=(k_interesse + n_exercicio*0.15, prob_k_interesse + max(probs_ex)*0.2),
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                fontsize=11, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.8))
    
    plt.xlabel('Número de Sucessos (k)')
    plt.ylabel('Probabilidade')
    plt.title(f'Distribuição Binomial - Seu Exercício\nn={n_exercicio}, p={p_exercicio}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 📝 Resumo e Conclusões

### ✅ O que aprendemos:

1. **Conceito**: A distribuição binomial modela experimentos com resultados binários repetidos
2. **Parâmetros**: n (tentativas) e p (probabilidade de sucesso)
3. **Propriedades**: Média = np, Variância = np(1-p)
4. **Aplicações**: Controle de qualidade, pesquisas, jogos, etc.
5. **Simulação**: A Lei dos Grandes Números confirma a teoria

### 🔍 Observações importantes:

- Quando **p = 0.5**, a distribuição é **simétrica**
- Quando **p < 0.5**, a distribuição é **assimétrica à direita**
- Quando **p > 0.5**, a distribuição é **assimétrica à esquerda**
- Quanto **maior n**, mais a distribuição se aproxima da **normal** (Teorema Central do Limite)

### 💡 Dicas práticas:

- Use a distribuição binomial quando tiver **experimentos independentes** com **dois resultados**
- Verifique sempre se a **probabilidade é constante** em todas as tentativas
- Para **n grande** e **p próximo de 0.5**, considere aproximar pela distribuição normal
- Para **n grande** e **p pequeno**, considere usar a distribuição de Poisson